# Structured Output with Autogen Agent
Structuring the output of Large Language Models (LLMs) offers numerous benefits, including improved accuracy, enhanced decision-making, and increased efficiency. Structured output enables models to capture subtle nuances and complexities in language, leading to more accurate predictions or classifications. Consistency is another crucial aspect of structured output for LLMs, offering advantages such as easier comparison, simplified analysis, improved debugging, enhanced reproducibility, and increased stability.

AutoGen provides a unified multi-agent conversation framework as a high-level abstraction for using foundation models. However, it does not offer an easy way to produce structured output. This notebook presents a workaround that utilizes agent tool calling to structure the output.

In [1]:
from openai import OpenAI
import json
from autogen import ConversableAgent, AssistantAgent, UserProxyAgent
from autogen import register_function
from typing_extensions import Annotated

config_list = [
    {
        "model": "mistral-nemo",
        "base_url": "http://localhost:11434/v1",
        "api_key": "ollama",
    }
]

In [2]:
from langchain_community.chat_models import ChatOllama
llm = ChatOllama(model="llama3.1", temperature=0.1)

In [3]:
# Create a tool that compels the LLM to generate structured data and validate it to ensure it conforms to the schema.
def get_structured_output(name: Annotated[str, "name"], height: Annotated[float, "height in cm"],
                          hair_color: Annotated[str, "hair color"],
                          age: Annotated[int, "age"]) -> Annotated[str, "Person info."]:
    res = {name: name, height: height, hair_color: hair_color, age: age}
    if name and height and hair_color and age:
        return 'The output structure is correct'
    else:
        return 'The output structure is incorrect'

def check_terminate(msg):
    print(msg)
    if '[TERMINATE]' in msg:
        return True
    else:
        return False

personal_info_extractor = ConversableAgent("personal_info_extractor",
                        system_message="You are a smart agent. "
                                    "You learned each person's height, hair color and age from the "
                                    "context and find the most matched person."
                                    "call get_structured_output(name, height, hair_color, age).",
                        llm_config={"config_list": config_list}, max_consecutive_auto_reply=1,
                        human_input_mode="NEVER",)
user_proxy = ConversableAgent("user_proxy", 
                        default_auto_reply="Did you get correct result from tool calls, answer [TERMINATE] if you got correct result.",
                        is_termination_msg=check_terminate)

register_function(
        get_structured_output,
        caller=personal_info_extractor,
        executor=user_proxy,
        name="get_structured_output",
        description="Get structured output.",
    )


In [4]:

response = user_proxy.initiate_chat(personal_info_extractor, message="Alex is a 36 year-old man who is 6 feet tall and has blonde hair. "
                                       "Claudia is a 25 year-old brunette woman who is 5 feet tall. Extract people from the sentence")


user_proxy (to personal_info_extractor):

Alex is a 36 year-old man who is 6 feet tall and has blonde hair. Claudia is a 25 year-old brunette woman who is 5 feet tall. Extract people from the sentence

--------------------------------------------------------------------------------
personal_info_extractor (to user_proxy):


***** Suggested tool call (call_gqjx1jqh): get_structured_output *****
Arguments: 
{"age":36,"hair_color":"blonde","height":182.88,"name":"Alex"}
**********************************************************************
***** Suggested tool call (call_p25jqyvc): get_structured_output *****
Arguments: 
{"age":25,"hair_color":"brunette","height":152.4,"name":"Claudia"}
**********************************************************************

--------------------------------------------------------------------------------
{'content': '', 'tool_calls': [{'id': 'call_gqjx1jqh', 'function': {'arguments': '{"age":36,"hair_color":"blonde","height":182.88,"name":"Alex"}', 'name':

[runtime logging] log_function_use: autogen logger is None



>>>>>>>> EXECUTING FUNCTION get_structured_output...


[runtime logging] log_function_use: autogen logger is None


user_proxy (to personal_info_extractor):

user_proxy (to personal_info_extractor):

***** Response from calling tool (call_gqjx1jqh) *****
The output structure is correct
******************************************************

--------------------------------------------------------------------------------
user_proxy (to personal_info_extractor):

***** Response from calling tool (call_p25jqyvc) *****
The output structure is correct
******************************************************

--------------------------------------------------------------------------------


In [5]:
response

ChatResult(chat_id=None, chat_history=[{'content': 'Alex is a 36 year-old man who is 6 feet tall and has blonde hair. Claudia is a 25 year-old brunette woman who is 5 feet tall. Extract people from the sentence', 'role': 'assistant'}, {'content': '', 'tool_calls': [{'id': 'call_gqjx1jqh', 'function': {'arguments': '{"age":36,"hair_color":"blonde","height":182.88,"name":"Alex"}', 'name': 'get_structured_output'}, 'type': 'function'}, {'id': 'call_p25jqyvc', 'function': {'arguments': '{"age":25,"hair_color":"brunette","height":152.4,"name":"Claudia"}', 'name': 'get_structured_output'}, 'type': 'function'}], 'role': 'assistant'}, {'content': 'The output structure is correct\n\nThe output structure is correct', 'tool_responses': [{'tool_call_id': 'call_gqjx1jqh', 'role': 'tool', 'content': 'The output structure is correct'}, {'tool_call_id': 'call_p25jqyvc', 'role': 'tool', 'content': 'The output structure is correct'}], 'role': 'tool'}], summary='The output structure is correct\n\nThe out

In [6]:
import json

def structure_output(response):
    structured_outputs = []
    for chat in response.chat_history:
        tool_calls = []
        # find last tools calls
        if 'tool_calls' in chat:
            tool_calls = chat['tool_calls']
        for tool_call in tool_calls:
            args = tool_call['function']['arguments']
            structured_output = json.loads(args)
            structured_outputs.append(structured_output)
    return structured_outputs

In [7]:
# found two people, Alex and Claudia with structured output
structure_output(response)

[{'age': 36, 'hair_color': 'blonde', 'height': 182.88, 'name': 'Alex'},
 {'age': 25, 'hair_color': 'brunette', 'height': 152.4, 'name': 'Claudia'}]